In [11]:
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, RMSprop
from resnet50 import ResNet50
from logger import logger
from keras.preprocessing import image
import numpy as np
from keras import backend as K
import datetime
import os
import cPickle as pickle

In [3]:
def load_pretrained_model(model_name, output_classes, weights_path='original'):
    # Load base model
    if model_name == 'resnet50' and weights_path == 'original':
        base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None)
        logger.info('Base model loaded with original weights')
    elif model_name == 'resnet50' and weights_path != 'original':
        base_model = ResNet50(include_top=False, weights=None, input_tensor=None)
        weights_path = weights_path
        logger.info('Base model loaded with weights in {}'.format(weights_path))
    else:
        raise Exception('Base model not loaded correctly')

    # Create top block
    x = base_model.output
    x = Flatten(name='flatten')(x)
    x = Dense(512, activation='relu', init='glorot_uniform', name='relu_1')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu', init='glorot_uniform', name='relu_2')(x)
    x = Dropout(0.5)(x)
    pred_layer = Dense(output_dim=output_classes, activation='softmax', name='softmax_output')(x)

    # Create overall model
    model = Model(input=base_model.input, output=pred_layer)

    # Load weights if not original weights
    if weights_path != 'original':
        model.load_weights(weights_path)

    return model


In [4]:
model = load_pretrained_model(model_name='resnet50', output_classes=65, weights_path='../data/images_clothes/model/resnet50_finetuned_4block.h5')

/Users/eugeneyan/.virtualenvs/datagene/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/Users/eugeneyan/.virtualenvs/datagene/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/Users/eugeneyan/.virtualenvs/datagene/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')
/Users/eugeneyan/.virtualenvs/datagene/lib/python2.7/site-packages/keras/backend/theano_backend.py:1505: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='average_

In [2]:
def create_category_dict(train_dir):
    categories = os.listdir(train_dir)
    
    # Initialize category dict
    category_dict = dict()
    idx = 0
    for category in categories:
        if category != '.DS_Store':
            category_dict[idx] = category
            idx += 1
    
    return category_dict

In [3]:
train_dir = '../data/images_clothes/train/'
category_dict = create_category_dict(train_dir)

In [12]:
def save_dict(tfidf_dict, int_to_category_dict, output_dir, output_name):
    """ (defaultdict, dict, str, str) -> NoneType

    Saves the dictionaries (tfidf_dict, int_to_category_dict) into pickle format

    :param tfidf_dict:
    :param int_to_category_dict:
    :param output_dir:
    :param output_name:
    :return:
    """
    output_dir_path = os.path.join(output_dir, output_name + '.pickle')

    with open(output_dir_path, 'wb') as handle:
        pickle.dump((tfidf_dict, int_to_category_dict), handle, protocol=2)
        logger.info('Dict saved in {}'.format(output_dir_path))


In [13]:
output_dir_path = os.path.join('../data/images_clothes/model/', 'image_category_dict' + '.pickle')

In [14]:
with open(output_dir_path, 'wb') as handle:
    pickle.dump(category_dict, handle, protocol=2)
    logger.info('Dict saved in {}'.format(output_dir_path))

2016-11-27 13:15:40,226 - Dict saved in ../data/images_clothes/model/image_category_dict.pickle
INFO:__log__:Dict saved in ../data/images_clothes/model/image_category_dict.pickle


In [15]:
def load_dict(dict_dir='categorize', dict_name='tfidf_dict'):
    """ (str, str) -> defaultdict

    Loads a dictionary for categorization into memory

    :param tfidf_dict:
    :param dict_dir:
    :param dict_name:
    :return:
    """
    output_dir_path = os.path.join(dict_dir, dict_name + '.pickle')

    with open(output_dir_path, 'rb') as handle:
        logger.info('Dictionary loading from: {}/{}.pickle'.format(dict_dir, dict_name))
        return pickle.load(handle)

In [17]:
x = load_dict('../data/images_clothes/model/', dict_name='image_category_dict')

2016-11-27 13:17:14,655 - Dictionary loading from: ../data/images_clothes/model//image_category_dict.pickle
INFO:__log__:Dictionary loading from: ../data/images_clothes/model//image_category_dict.pickle


In [18]:
x

{0: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Active -> Active Shirts & Tees',
 1: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Active -> Active Shorts',
 2: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Active -> Athletic Socks',
 3: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Fashion Hoodies & Sweatshirts',
 4: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Jackets & Coats -> Leather & Faux Leather',
 5: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Jeans',
 6: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Pants -> Casual',
 7: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Pants -> Dress',
 8: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Shirts -> Casual Button-Down Shirts',
 9: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Shirts -> Dress Shirts',
 10: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Shirts -> Polos',
 11: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Shirts -> T-Shirts',
 12: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> S

### Image Class

In [84]:
pred_image_path = '../data/images_clothes/pred_image/B000KPX728.jpg'

In [85]:
pred_image=image.load_img(pred_image_path, target_size=(224, 224))
pred_image=image.img_to_array(pred_image)
pred_image=np.multiply(pred_image, 1./255)
pred_image=np.expand_dims(pred_image, axis=0)

In [87]:
start_time = datetime.datetime.now()
preds = model.predict(pred_image)
end_time = datetime.datetime.now()
elapsed_time = end_time - start_time
print('Time taken: {} secs'.format(elapsed_time.total_seconds()))

Time taken: 121.990858 secs


In [113]:
preds

array([[  1.53738940e-02,   8.50417564e-06,   1.10553874e-05,
          2.88268493e-04,   1.10323060e-06,   3.42248910e-07,
          2.05393621e-06,   3.88642235e-07,   8.75924539e-04,
          1.34680067e-05,   4.12318295e-05,   6.29196286e-01,
          3.85864816e-08,   1.20124241e-06,   1.79214567e-05,
          3.51466656e-06,   4.67137443e-06,   3.16086744e-06,
          2.29888497e-06,   1.10631754e-06,   2.71122275e-07,
          1.96324047e-02,   1.32565810e-06,   1.22797047e-07,
          3.68912509e-08,   4.43729135e-04,   2.11100341e-05,
          4.33262312e-06,   5.07978484e-06,   3.26003821e-04,
          1.99041196e-07,   1.74347642e-05,   1.29935438e-06,
          2.72796569e-06,   1.15653745e-06,   2.51497090e-06,
          8.77074672e-06,   1.79756921e-06,   3.62733573e-07,
          3.53415799e-03,   1.45768034e-04,   5.25666052e-04,
          1.23343580e-06,   7.94365320e-08,   1.87987350e-08,
          5.40643100e-07,   2.50541439e-06,   3.02270314e-06,
        

In [91]:
category_dict

{0: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Active -> Active Shirts & Tees',
 1: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Active -> Active Shorts',
 2: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Active -> Athletic Socks',
 3: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Fashion Hoodies & Sweatshirts',
 4: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Jackets & Coats -> Leather & Faux Leather',
 5: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Jeans',
 6: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Pants -> Casual',
 7: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Pants -> Dress',
 8: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Shirts -> Casual Button-Down Shirts',
 9: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Shirts -> Dress Shirts',
 10: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Shirts -> Polos',
 11: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> Shirts -> T-Shirts',
 12: 'Clothing, Shoes & Jewelry -> Men -> Clothing -> S

In [60]:
top=5

In [123]:
top_indices = (-preds).argsort()[:, :top][0]

In [124]:
top_indices

array([11, 57, 21,  0, 39])

In [125]:
preds[0][11]

0.62919629

In [126]:
category_dict[11]

'Clothing, Shoes & Jewelry -> Men -> Clothing -> Shirts -> T-Shirts'

In [145]:
results = dict()

In [146]:
for i, idx in enumerate(top_indices):
    category = category_dict[idx]
    prob = preds[0][idx]
    results[i] = (category, prob)

In [147]:
results

{0: ('Clothing, Shoes & Jewelry -> Men -> Clothing -> Shirts -> T-Shirts',
  0.62919629),
 1: ('Clothing, Shoes & Jewelry -> Women -> Clothing -> Tops & Tees -> Knits & Tees',
  0.32668123),
 2: ('Clothing, Shoes & Jewelry -> Women -> Clothing -> Active -> Active Shirts & Tees',
  0.019632405),
 3: ('Clothing, Shoes & Jewelry -> Men -> Clothing -> Active -> Active Shirts & Tees',
  0.015373894),
 4: ('Clothing, Shoes & Jewelry -> Women -> Clothing -> Lingerie, Sleep & Lounge -> Sleep & Lounge -> Nightgowns & Sleepshirts',
  0.003534158)}

In [ ]:
start_time = datetime.datetime.now()
preds = model.predict(pred_image)
end_time = datetime.datetime.now()
elapsed_time = end_time - start_time
print('Time taken: {} secs'.format(elapsed_time.total_seconds()))

In [172]:
class Image:
    
    def __init__(self, image_path):
        self.image_path = image_path
        self.image = None
        logger.debug('Image received')
        
    def prepare(self):
        self.image = image.load_img(self.image_path, target_size=(224, 224))
        self.image = image.img_to_array(self.image)
        self.image = np.multiply(self.image, 1./255)
        self.image = np.expand_dims(self.image, axis=0)
        logger.debug('Image prepared')
        
        return self
    
    def categorize(self):
        start_time = datetime.datetime.now()
        
        preds = model.predict(self.image)
        
        end_time = datetime.datetime.now()
        elapsed_time = end_time - start_time
        elapsed_time = elapsed_time.total_seconds()
        logger.info('Time taken: {} secs'.format(elapsed_time))
        
        top = 5
        results = dict()
        
        top_indices = (-preds).argsort()[:, :top][0]
        for i, idx in enumerate(top_indices):
            category = category_dict[idx]
            prob = preds[0][idx]
            results[i] = (category, prob)
            
        return results, elapsed_time

In [168]:
def image_categorize_single(image_path):
    return Image(image_path).prepare().categorize()

In [170]:
pred_image_path = '../data/images_clothes/pred_image/B0000C2QZX.jpg'

In [166]:
Image(pred_image_path).prepare().categorize()

DEBUG:__log__:Image received
DEBUG:__log__:Image prepared
2016-11-27 09:50:06,023 - Time taken: 118.217844 secs
INFO:__log__:Time taken: 118.217844 secs


({0: ('Clothing, Shoes & Jewelry -> Women -> Clothing -> Shorts -> Denim',
   0.58311218),
  1: ('Clothing, Shoes & Jewelry -> Women -> Clothing -> Shorts -> Casual',
   0.36631334),
  2: ('Clothing, Shoes & Jewelry -> Men -> Clothing -> Underwear -> Boxer Briefs',
   0.03935191),
  3: ('Clothing, Shoes & Jewelry -> Men -> Clothing -> Swim -> Board Shorts',
   0.0065661236),
  4: ('Clothing, Shoes & Jewelry -> Women -> Clothing -> Skirts -> Casual',
   0.0010080608)},
 datetime.timedelta(0, 118, 217844))

In [173]:
image_categorize_single(pred_image_path)

DEBUG:__log__:Image received
DEBUG:__log__:Image prepared
2016-11-27 09:53:18,726 - Time taken: 119.335935 secs
INFO:__log__:Time taken: 119.335935 secs


({0: ('Clothing, Shoes & Jewelry -> Men -> Clothing -> Underwear -> Briefs',
   0.98395598),
  1: ('Clothing, Shoes & Jewelry -> Men -> Clothing -> Underwear -> Boxer Briefs',
   0.016025206),
  2: ('Clothing, Shoes & Jewelry -> Women -> Clothing -> Lingerie, Sleep & Lounge -> Intimates -> Panties -> Briefs',
   1.8811246e-05),
  3: ('Clothing, Shoes & Jewelry -> Men -> Clothing -> Active -> Active Shorts',
   3.079011e-09),
  4: ('Clothing, Shoes & Jewelry -> Women -> Clothing -> Shorts -> Casual',
   9.0954827e-10)},
 119.335935)